In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D


In [8]:
import zipfile
with zipfile.ZipFile('chest-ctscan-images.zip','r') as zip_ref:
  zip_ref.extractall('chest-ctscan-images')

In [9]:
train_data = ImageDataGenerator(rescale=1./255,validation_split=0.2,rotation_range=5,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')
validation_data = ImageDataGenerator(rescale= 1./255,validation_split=0.2)
test_data = ImageDataGenerator(rescale=1./255)

In [10]:
train_dataset = train_data.flow_from_directory(directory='/content/chest-ctscan-images/Data/train',target_size=(224,224),class_mode='categorical',batch_size=64)

Found 613 images belonging to 4 classes.


In [13]:
test_dataset = test_data.flow_from_directory(directory='/content/chest-ctscan-images/Data/test',target_size=(224,224),class_mode='categorical',batch_size=64)

Found 315 images belonging to 4 classes.


In [14]:
valid_dataset = validation_data.flow_from_directory(directory='/content/chest-ctscan-images/Data/valid',target_size=(224,224),class_mode='categorical',batch_size=64)

Found 72 images belonging to 4 classes.


In [15]:
from keras.applications import VGG19
vgg = VGG19(input_shape = (224,224,3),include_top = False,weights="imagenet")


80134624/80134624 [==============================] - 0s 0us/step


In [27]:
for layer in vgg.layers:
 vgg.trainable = False

In [29]:
x = Flatten()(vgg.output)
pred= Dense(4,activation='softmax')(x)
model = keras.models.Model(inputs= vgg.input,outputs = pred)

In [30]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [31]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

In [32]:
hist = model.fit(train_dataset,validation_data=valid_dataset,epochs=10)

Epoch 1/10
10/10 [==============================] - 412s 41s/step - loss: 2.5034 - accuracy: 0.3458 - val_loss: 1.2826 - val_accuracy: 0.4028
Epoch 2/10
10/10 [==============================] - 404s 41s/step - loss: 1.2816 - accuracy: 0.5106 - val_loss: 1.1769 - val_accuracy: 0.4167
Epoch 3/10
10/10 [==============================] - 404s 41s/step - loss: 0.9736 - accuracy: 0.5677 - val_loss: 0.9685 - val_accuracy: 0.5417
Epoch 4/10
10/10 [==============================] - 406s 41s/step - loss: 0.8988 - accuracy: 0.6052 - val_loss: 1.1884 - val_accuracy: 0.4306
Epoch 5/10
10/10 [==============================] - 404s 41s/step - loss: 0.8445 - accuracy: 0.6085 - val_loss: 0.9825 - val_accuracy: 0.5278
Epoch 6/10
10/10 [==============================] - 403s 41s/step - loss: 0.7986 - accuracy: 0.6395 - val_loss: 1.1689 - val_accuracy: 0.4861
Epoch 7/10
10/10 [==============================] - 408s 41s/step - loss: 0.7543 - accuracy: 0.6558 - val_loss: 1.0752 - val_accuracy: 0.5139
Epoch 

In [33]:
model.evaluate(test_dataset,verbose=1)

5/5 [==============================] - 186s 37s/step - loss: 0.9818 - accuracy: 0.6413


[0.9818202257156372, 0.641269862651825]